# Blocks in Ruby

Ruby Code blocks (called _closures_ in other languages) are definitely one of the coolest features of Ruby and are chunks of code between braces or between `do..end` that you can associate with _method_ invocations, almost as if they were parameters. A Ruby block is a way of grouping statemetns, and may appear only in the source _adjacent to a method call_; the block is written starting on the same line as the method call's last parameter (or the closing parenthesis of the parameter list). The code in the block is not executed at the time it is encountered. Instead, Ruby remembers the context in which the blocks appears (the local variables, the current object, and so on) and then enters the method.

The Ruby standard is to use braces for single-line blocks and `do..end` for multi-line blocks. Keep in mind that the braces syntax has a higher precedence than the `do..end` syntax. Braces have a high precedence; `do` has a low precedence. If the method invocation has parameters that are not enclosed in parentheses, the brace form of a block will bind to the last parameter, not to the overall invocation. The `do` form will bind to the invocation.

Matz says that _any method_ can be called with blocks as an implicit argument. Inside the method, you can call the block using the `yield` keyword with a value.

Also, as you will soon learn, blocks can have their own arguments. There are many methods in Ruby that iterate over a range of values. Most of these iterators are written in such a way as to be able to take a code block as part of the calling syntax. The method can then yiled control to the code block (i.e. execute the block) during execution as many times as is necessary for the iteration to complete (e.g. if we are iterating over array values, we can execute the block as many times as there are array values etc.).

Once you have created a block, you can associate it with a call to a method. Usually the code blocks passed into methods are anonymous objects, created on the spot. For example, in the following code, the block containing `puts "Hello"` is associated with the call to a method `greet`.

In [1]:
def greet
    yield
end
greet {puts "Hello"}

Hello


If the method has parameters, they appear before the block.

In [2]:
def verbose_greet(info)
    puts info
    yield
end
verbose_greet("PuneRuby") {puts 'Hello'}

PuneRuby
Hello


A method can then invoke an _associated_ block one or more times using the Ruby `yield` statement.

Thus any method that wants to take a block as a parameter can use the `yield` keyword to execute the block at any time.

In [3]:
def call_block
    puts 'Start of method'
    yield
    yield
    puts 'End of method'
end
call_block {puts 'In the block'}

Start of method
In the block
In the block
End of method


If you provide a code block when you call a method, then inside the method, you can yield control to that code block - suspend execution of the method; execute the code in the block; and return control to the method body, right after the call to `yield`. If no code block is passed and `yield` is called, Ruby raises an exception:

```
no block given (LocalJumpError)
```

You can provide parameters to the call to `yield`: these will be passed to the block. Within the block, you list the names of the arguments to receive the parameters between vertical bars (`|`).

In [4]:
def call_block
    yield('hello', 99)
end
call_block {|str, num| puts str + ' ' + num.to_s}

hello 99


**Note** that the code in the block is not executed at the time it is encountered by the Ruby interpreter. Instead, Ruby remembers the context in which the block appears and then enters the method.

A code block's return value (like that of a method) is the value of the last expression evaluated in the code block. This return valur is made available inside the method; it comes through as the return value of `yield`.

`block_given?` returns `true` if the function is passed with a block which can be yielded.

In [5]:
def try
    if block_given?
        yield
    else
        puts 'no block'
    end
end
try
try {puts 'hello'}
try do puts 'bye' end

no block
hello
bye


## Block Variables

Let us see what happens in the following example when a variable outside a block is `x` and a block parameter is also named `x`.

In [6]:
x = 10
5.times do |x|
    puts "x inside the block: #{x}"
end
puts "x outside the block: #{x}"

x inside the block: 0
x inside the block: 1
x inside the block: 2
x inside the block: 3
x inside the block: 4
x outside the block: 10


You will observe that after the block has executed, `x` outside the block is the original `x`. Hence the block parameter `x` was local to the block.

Next observe what happens to `x` in the following example:

In [7]:
x = 10
5.times do |y|
    x = y
    puts "x inside the block: #{x}"
end
puts "x outside the block: #{x}"

x inside the block: 0
x inside the block: 1
x inside the block: 2
x inside the block: 3
x inside the block: 4
x outside the block: 4


Since `x` is _not a block parameter_ here, the variable `x` is the same inside and outside the block.

From Ruby 1.9 on-wards, blocks introduced their own scope for the _block parameters only_. This is illustrated by the following example:

In [8]:
x = 10
5.times do |y; x|
    x = y
    puts "x inside the block: #{x}"
end
puts "x outside the block: #{x}"

x inside the block: 0
x inside the block: 1
x inside the block: 2
x inside the block: 3
x inside the block: 4
x outside the block: 10


In the above block, a new feature is being used: block local variable. In short, block local variables shield a block from manipulating variables outside of its scope. This prevents a block from unintencionally clobbering any variables outside its scope. If you don't want to clobber variables, use block local variables for the variables your block creates.

The syntax for a block local variable is simple. Put a semicolon after the normal block parameter list, then list the variable you want as block local variables. For example, if the block takes two variables `a` and `b`, and uses to local variable `x` and `y`, the parameter list would look like this: `|a,b;x,y|`.